In [ ]:
from analysis.paper import *

In [ ]:
ags = data.load()
ags = data.with_times(ags)

In [ ]:
upload(plot_flops_curves, ags)

In [ ]:
plot_frontiers(ags)

In [ ]:
overleaf.table(max_sizes_table(ags), 'maxsizes')

In [ ]:
(ags
        .groupby('boardsize')
        [['width', 'depth', 'samples', 'train_flops']]
        .max()
        .assign(train_flops=lambda df: df.train_flops.apply(lambda s: f'{s:.0G}'))
        .assign(samples=lambda df: df.samples.apply(lambda s: f'{s:.0G}'))
        .rename(columns={'boardsize': 'board size', 'width': 'max neurons', 'depth': 'max layers', 'samples': 'max samples', 'train_flops': 'max flops'})
        .reset_index())

In [ ]:
overleaf.table(boardsize_hyperparams_table(ags), 'boardsize_hyperparams')

In [ ]:
df = ags.query('test_nodes == 64').copy()
df['g'] = ags.run
labels = df.sort_values('train_flops').groupby('boardsize').first().reset_index()

(pn.ggplot(df, pn.aes(x='train_flops', color='factor(boardsize)', group='g'))
    + pn.geom_line(pn.aes(y='400/np.log(10)*elo'), size=.25, show_legend=False)
    + pn.geom_point(pn.aes(y='400/np.log(10)*elo'), size=1/16, show_legend=False, shape='.')
    + pn.geom_text(pn.aes(y='400/np.log(10)*elo', label='boardsize'), data=labels, show_legend=False, size=6, nudge_x=-.25, nudge_y=-15)
    + pn.labs(
        x='FLOPS', 
        y='Elo v. perfect play')
    + pn.scale_color_discrete(l=.4)
    + pn.scale_x_continuous(trans='log10')
    + pn.coord_cartesian(None, (None, 0))
    + plot.IEEE())

In [ ]:
labels